### DIGIT RECOGNISER NEURAL NETWORK
(using only NumPy)

In this notebook, I aim to implement a basic two-layer neural network and trained it on the MNIST digit recogniser dataset which contains sample hand-written digits of 28 x 28 pixel resolution. This was largely done with the help of Samson Zhang's YouTube tutorial.

In [10]:
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt 

data = pd.read_csv('train.csv')

In [8]:
data = np.array(data)
m, n = data.shape

#shuffles the databefore splitting it into dev and training sets
np.random.shuffle(data)

data_dev = data[0:1000].T 
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T 
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_, m_train = X_train.shape


In [9]:
Y_train

array([7, 0, 8, ..., 3, 9, 1], dtype=int64)

The Neural Network will have a two-layer architecture. The input layer $a^{[0]}$ will have 784 units corresponding to the pixels in the input image (28 x 28 pixels). The hidden layer $a^{[1]}$ will have 10 units with ReLU activation, and the output layer $a^{[2]}$ will have 10 units corresponding to the ten digit classes we aim to classify the image into, with softmax activation.


## The Math

**Forward Propagation**

$$Z^{[1]} = W^{[1]}X + b^{[1]}$$
$$A^{[1]} = g_{\text{ReLU}}(Z^{[1]})$$
$$Z^{[2]} = W^{[2]}A^{[1]} + b^{[2]}$$
$$A^{[2]} = g_{\text{softmax}}(Z^{[2]})$$

**Backward propagation**

$$dZ^{[2]} = A^{[2]} - Y$$
$$dW^{[2]} = \frac{1}{m} dZ^{[2]} A^{[1]T}$$
$$dB^{[2]} = \frac{1}{m} \Sigma {dZ^{[2]}}$$
$$dZ^{[1]} = W^{[2]T} dZ^{[2]} .* g^{[1]\prime} (z^{[1]})$$
$$dW^{[1]} = \frac{1}{m} dZ^{[1]} A^{[0]T}$$
$$dB^{[1]} = \frac{1}{m} \Sigma {dZ^{[1]}}$$

**Parameter updates**

$$W^{[2]} := W^{[2]} - \alpha dW^{[2]}$$
$$b^{[2]} := b^{[2]} - \alpha db^{[2]}$$
$$W^{[1]} := W^{[1]} - \alpha dW^{[1]}$$
$$b^{[1]} := b^{[1]} - \alpha db^{[1]}$$



In [11]:
def init_parameters():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def SoftMax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = SoftMax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [12]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_parameters()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 50 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [13]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 500)

Iteration:  0
[6 6 1 ... 2 3 2] [7 0 8 ... 3 9 1]
0.08790243902439024
Iteration:  10
[6 6 3 ... 2 3 2] [7 0 8 ... 3 9 1]
0.13609756097560977
Iteration:  20
[6 6 1 ... 2 3 2] [7 0 8 ... 3 9 1]
0.2068780487804878
Iteration:  30
[6 6 1 ... 2 3 1] [7 0 8 ... 3 9 1]
0.2578780487804878
Iteration:  40
[6 6 1 ... 8 3 1] [7 0 8 ... 3 9 1]
0.29236585365853657
Iteration:  50
[6 0 2 ... 8 3 1] [7 0 8 ... 3 9 1]
0.3238780487804878
Iteration:  60
[6 0 2 ... 8 3 1] [7 0 8 ... 3 9 1]
0.3550243902439024
Iteration:  70
[6 0 8 ... 8 3 1] [7 0 8 ... 3 9 1]
0.3893658536585366
Iteration:  80
[0 0 8 ... 6 6 1] [7 0 8 ... 3 9 1]
0.4200243902439024
Iteration:  90
[0 0 8 ... 6 6 1] [7 0 8 ... 3 9 1]
0.45021951219512196
Iteration:  100
[0 0 8 ... 6 6 1] [7 0 8 ... 3 9 1]
0.4804390243902439
Iteration:  110
[0 0 8 ... 6 6 1] [7 0 8 ... 3 9 1]
0.5123414634146342
Iteration:  120
[0 0 8 ... 6 6 1] [7 0 8 ... 3 9 1]
0.5406829268292683
Iteration:  130
[0 0 8 ... 6 6 1] [7 0 8 ... 3 9 1]
0.5626585365853658
Iteration:  1

KeyboardInterrupt: 